# Predicciones triviales para productos objetivo – Subida inicial a Kaggle

Este notebook tiene como objetivo generar predicciones **triviales** para un conjunto de productos definidos por el profesor y mentor de la materia.  
Estas predicciones no utilizan modelos estadísticos ni de machine learning y sirven como línea base para evaluación en la competencia de Kaggle.

## Predicciones generadas

1. Último valor observado (mes `201912`)
2. Promedios simples de:
   - Últimos **3 meses** (`201910` a `201912`)
   - Últimos **6 meses** (`201907` a `201912`)
   - Últimos **9 meses** (`201904` a `201912`)
   - Últimos **12 meses** (`201901` a `201912`)

## Formato de salida

Cada predicción se exporta como un archivo CSV con las columnas:
- `product_id`
- `tn` (valor predicho en toneladas)

El nombre del archivo describe la ventana temporal, por ejemplo:
- `pred_trivial_last_201912.csv`
- `pred_trivial_avg_3m_201910_201912.csv`


## Librerías necesarias

In [11]:

import pandas as pd
import numpy as np


## Carga de datos y lista de productos objetivo

In [12]:

# Archivos fuente
DATA_PATH = r"C:\Developer\Laboratorio_III\data"
SELLIN_FILE = DATA_PATH + r"\sell-in.txt"
PRODUCTOS_KAGGLE_FILE = DATA_PATH + r"\product_id_apredecir201912.txt"  # lista proporcionada por el profesor

# Carga
df_sellin = pd.read_csv(SELLIN_FILE, sep='\t')
df_productos_kaggle = pd.read_csv(PRODUCTOS_KAGGLE_FILE)

# Asegurar tipo string para 'periodo'
df_sellin['periodo'] = df_sellin['periodo'].astype(str)


## Función para generar predicción y guardar CSV

In [13]:

def generar_prediccion(df, productos, desde, hasta=None, metodo='avg'):
    '''
    df: dataframe de sell-in
    productos: dataframe con columna product_id
    desde, hasta: strings con formato YYYYMM
    metodo: 'avg' o 'last'
    '''
    if metodo == 'last':
        df_filtrado = df[df['periodo'] == desde]
        pred = df_filtrado.groupby('product_id')['tn'].sum().reset_index()
        nombre_archivo = f"pred_trivial_last_{desde}.csv"

    elif metodo == 'avg':
        df_filtrado = df[(df['periodo'] >= desde) & (df['periodo'] <= hasta)]
        # Paso 1: sumar ventas por producto y periodo
        ventas_mensuales = df_filtrado.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()
        # Paso 2: promedio de esas ventas mensuales por producto
        pred = ventas_mensuales.groupby('product_id')['tn'].mean().reset_index()
        nombre_archivo = f"pred_trivial_avg_{desde}_{hasta}.csv"


    else:
        raise ValueError("Método no válido. Usar 'avg' o 'last'.")

    # Filtrar solo productos de la lista
    pred = pred[pred['product_id'].isin(productos['product_id'])]

    # Guardar
    pred[['product_id', 'tn']].to_csv(nombre_archivo, index=False)
    print(f"Archivo generado: {nombre_archivo}")


## Generación de predicciones y guardado

In [ ]:

# Último valor observado
#generar_prediccion(df_sellin, df_productos_kaggle, desde="201912", metodo='last')

# Promedios móviles
#generar_prediccion(df_sellin, df_productos_kaggle, desde="201910", hasta="201912", metodo='avg')
#generar_prediccion(df_sellin, df_productos_kaggle, desde="201907", hasta="201912", metodo='avg')
#generar_prediccion(df_sellin, df_productos_kaggle, desde="201904", hasta="201912", metodo='avg')
generar_prediccion(df_sellin, df_productos_kaggle, desde="201901", hasta="201912", metodo='avg')


Archivo generado: pred_trivial_avg_201807_201912.csv
